In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Dep, DepReport
from typing import Dict
import pickle


with open(OUTPUT_PATH / "bcc.pkl", "rb") as f:
    data: Dict[str, Dict[Dep, DepReport]] = pickle.load(f)

data.keys()

dict_keys(['bashreadline', 'bindsnoop', 'biolatency', 'biopattern', 'biosnoop', 'biostacks', 'biotop', 'bitesize', 'cachestat', 'capable', 'cpudist', 'cpufreq', 'drsnoop', 'execsnoop', 'exitsnoop', 'filelife', 'filetop', 'fsdist', 'fsslower', 'funclatency', 'futexctn', 'gethostlatency', 'hardirqs', 'javagc', 'klockstat', 'ksnoop', 'llcstat', 'mdflush', 'memleak', 'mountsnoop', 'numamove', 'offcputime', 'oomkill', 'opensnoop', 'profile', 'readahead', 'runqlat', 'runqlen', 'runqslower', 'sigsnoop', 'slabratetop', 'softirqs', 'solisten', 'statsnoop', 'syncsnoop', 'syscount', 'tcpconnect', 'tcpconnlat', 'tcplife', 'tcppktlat', 'tcprtt', 'tcpstates', 'tcpsynbl', 'tcptop', 'tcptracer', 'vfsstat', 'wakeuptime'])

In [3]:
data["Summary"] = {
    dep: report for deps in data.values() for dep, report in deps.items()
}

In [4]:
from depsurf import IssueEnum, DepKind, TAB_PATH
from collections import defaultdict
import pandas as pd

results = {}

KEY_NUMBER = "Count"

for prog_name, deps in data.items():
    if len(deps) == 0:
        print(f"Program {prog_name} has no dependencies")
        continue

    result = defaultdict(int)
    for dep, report in deps.items():
        kind = dep.kind
        result[(kind, KEY_NUMBER)] += 1

        num_absent = sum(
            1 for e in report.status.values() if IssueEnum.ABSENT in e.enums
        )
        result[(kind, IssueEnum.ABSENT)] += min(num_absent, 1)

        if kind != DepKind.STRUCT:
            num_delta = sum(1 for e in report.delta.values() if e.changes)
            result[(kind, IssueEnum.CHANGE)] += min(num_delta, 1)

        if kind == DepKind.FUNC:
            for issue in [
                IssueEnum.PARTIAL_INLINE,
                IssueEnum.FULL_INLINE,
                IssueEnum.RENAME,
                IssueEnum.DUPLICATE,
            ]:
                num_issue = sum(1 for e in report.status.values() if issue in e.enums)
                result[(kind, issue)] += min(num_issue, 1)

    has_issue = False
    for (kind, key), count in result.items():
        if key == KEY_NUMBER:
            continue
        if count != 0:
            has_issue = True
            break

    if not has_issue:
        prog_name = prog_name + r" \checkmark"

    results[prog_name] = result

df = pd.DataFrame(results).T.fillna(0).astype(int)

column_order = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.FIELD,
    DepKind.TRACEPOINT,
    DepKind.SYSCALL,
]
df = df.reindex(sorted(df.columns, key=lambda x: x[0]), axis=1)


def rename_cols(col):
    kind, key = col
    if kind in (DepKind.STRUCT, DepKind.FIELD):
        key = {
            (DepKind.STRUCT, KEY_NUMBER): "Count S",
            (DepKind.FIELD, KEY_NUMBER): "Count F",
            (DepKind.STRUCT, IssueEnum.ABSENT): "Absent S",
            (DepKind.FIELD, IssueEnum.ABSENT): "Absent F",
            (DepKind.FIELD, IssueEnum.CHANGE): "Changed F",
        }[col]
        kind = "Struct / Field"
    if kind == DepKind.TRACEPOINT:
        kind = "Tracept"
    if kind == DepKind.SYSCALL:
        kind = "Syscall"
    return (kind, key)


df.columns = df.columns.map(rename_cols)
df = df.sort_values(by=df.columns.tolist(), ascending=False)
df_non_summary = df.loc[df.index != "Summary"]
df = pd.concat([df_non_summary, df.loc[["Summary"]]])
df.loc[r"\textbf{\# tools}"] = df_non_summary.map(lambda x: min(x, 1)).sum(axis=0)
# df.loc["Pct. Tools"] = (df.loc["No. Tools"] / len(df_non_summary) * 100).astype(int)


df = df.replace(0, r"\color{lightgray}{-}")
df.columns = df.columns.map(lambda x: (x[0], f"\\head{{{x[1]}}}"))
with open(TAB_PATH / "bcc.tex", "w") as f:
    latex = df.to_latex(
        multicolumn_format="c|",
        column_format="l|rrrrrrr|rr|rrr|rrr|rrr",
    )
    latex = latex.replace(r"{c|}{Syscall}", r"{c}{Syscall}")
    latex = latex.replace(r"Summary", r"\midrule" + "\n" + r"\textbf{\# uniq deps}")
    f.write(latex)

# df

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.


Program bashreadline has no dependencies
Program funclatency has no dependencies
Program gethostlatency has no dependencies
Program javagc has no dependencies
Program ksnoop has no dependencies
